In [2]:
import os 

In [1]:
%pwd

'/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG/research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG'

# 1) Class entity step (--> src/entity/cong_entity.py)

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate : float

# 2) Configs manager step (--> src/config/configuration.py)

In [23]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE 
        )

        return training_config

# 3) Component definition step (--> src/components/name_of_the_step)

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [26]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(self.config.params_learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )


    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

# 4) Pipeline step (src/pipeline/stage_number)

In [29]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-01-25 19:58:17,133: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-25 19:58:17,135: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-25 19:58:17,135: INFO: common: created directory at: artifacts]
[2026-01-25 19:58:17,136: INFO: common: created directory at: artifacts/training]
[2026-01-25 19:58:17,196: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 694ms/step - accuracy: 0.7444 - loss: 7.9392 - val_accuracy: 0.8250 - val_loss: 1.1236
Epoch 2/5
 1/23 ━━━━━━━━━━━━━━━━━━━━ 12s 574ms/step - accuracy: 0.8750 - loss: 1.1451

/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG/.env/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - accuracy: 0.8750 - loss: 1.1451 - val_accuracy: 0.9625 - val_loss: 0.5412
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 731ms/step - accuracy: 0.8989 - loss: 1.0382 - val_accuracy: 1.0000 - val_loss: 1.5867e-05
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.9375 - loss: 1.0470 - val_accuracy: 1.0000 - val_loss: 5.5136e-05
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 745ms/step - accuracy: 0.9691 - loss: 0.3944 - val_accuracy: 1.0000 - val_loss: 7.7503e-04
[2026-01-25 19:59:14,646: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
